# @chain과 async/await로 비동기 체인 구성하기

이 노트북에서는 **@chain 데코레이터**와 **async/await**를 결합하여 비동기(Asynchronous) 체인을 구성하는 방법을 알아봅니다.

## 동기 vs 비동기

| 방식 | 메서드 | 특징 |
|------|--------|------|
| **동기 (Sync)** | `invoke()`, `batch()`, `stream()` | 순차 실행, 완료까지 대기 |
| **비동기 (Async)** | `ainvoke()`, `abatch()`, `astream()` | 동시 실행, I/O 대기 중 다른 작업 가능 |

## 비동기의 장점

1. **높은 처리량**: 여러 요청을 동시에 처리 (웹 서버에서 유용)
2. **효율적인 I/O**: 네트워크 대기 시간 동안 다른 작업 수행
3. **확장성**: FastAPI, aiohttp 등 비동기 프레임워크와 호환

## @chain + async 문법

```python
@chain
async def my_chain(values):         # async def로 정의
    prompt = await template.ainvoke(values)  # await + ainvoke()
    return await model.ainvoke(prompt)       # await + ainvoke()

# 호출 시에도 await + ainvoke()
result = await my_chain.ainvoke(inputs)
```

**핵심 규칙:**
- 함수: `async def`
- 내부 호출: `await` + `ainvoke()` / `abatch()` / `astream()`
- 외부 호출: `await` + `ainvoke()`

---

# 1. Ollama 설치 및 서버 실행

In [1]:
import subprocess
import time

# zstd 설치 (Ollama 설치의 사전 요구 사항)
!apt-get install -y zstd

# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# 백그라운드에서 Ollama 서버 실행
subprocess.Popen(['ollama', 'serve'])

time.sleep(3)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 2s (287 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current use

# 2. 모델 다운로드 & 패키지 설치

- `ollama pull llama3.2` - Llama 3.2 모델 다운로드
- `pip install langchain-ollama` - LangChain Ollama 통합 패키지 설치

In [2]:
!ollama pull llama3.2
!pip install -q langchain-ollama

# 3. 구성 요소 준비

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

# Prompt Template
template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('human', '{question}'),
])

# Model
model = ChatOllama(model='llama3.2')

# 4. 비동기 체인 정의

**코드 설명:**

### async def + @chain
```python
@chain
async def chatbot(values):
    prompt = await template.ainvoke(values)  # 비동기 프롬프트 생성
    return await model.ainvoke(prompt)        # 비동기 모델 호출
```

**주의사항:**
- 동기 메서드(`invoke`)가 아닌 **비동기 메서드(`ainvoke`)** 사용
- 각 비동기 호출 앞에 **`await`** 필수
- Jupyter 노트북에서는 `await`를 셀에서 직접 사용 가능

In [4]:
from langchain_core.runnables import chain

# 비동기 체인 정의
@chain
async def chatbot(values):
    prompt = await template.ainvoke(values)  # await + ainvoke
    return await model.ainvoke(prompt)        # await + ainvoke

# 5. 비동기 체인 실행

Jupyter 노트북에서는 `await`를 직접 사용할 수 있습니다.

In [5]:
# Jupyter에서 비동기 실행
response = await chatbot.ainvoke({'question': '거대 언어 모델은 어디서 제공하나요?'})

print("=== 비동기 실행 결과 ===")
print(response.content)

=== 비동기 실행 결과 ===
large language models을 제공하는 여러 곳이 있습니다.

1. **OpenAI**: OpenAI에서 제공하는 Large Language Model 5 (LLM-5)과 Large Language Model 6 (LLM-6)는 가장 유명한 examples입니다. 이 models은 비공식적인 API를 통해 사용할 수 있으며, 다양한 task에用于될 수 있습니다.
2. **Hugging Face**: Hugging Face의 Transformers library는 large language models을 제공합니다. 이 library는 open-source로, 다양한 task에 사용할 수 있으며, Ease of use가ูง하므로 많은 사람들의 favorites입니다.
3. **Google**: Google에서 개발한 BERT (Bidirectional Encoder Representations from Transformers)는 Large Language Models을 포함한 다양한 natural language processing(NLP) models을 제공합니다. 이 models은 Google의 cloud platform에 hosted되어 있지만, open-source로 사용할 수 있습니다.
4. **Microsoft**: Microsoft는large language models을 위한 Azure Cognitive Services를 제공합니다. 이 services는 cloud-based로, 다양한 task에 사용할 수 있으며, Ease of use가高합니다.

이 models은 각각 유사한 기능을 가지고 있지만, Each has its own strengths and weaknesses. Large language models을 사용하는 것에서 가장 좋은 방법은 각 models의 capabilities를 carefully evaluate하고, appropriate model을 선택하여 task에 best fit하도록

# 6. 여러 요청 동시 처리 (비동기의 진정한 힘)

`asyncio.gather()`를 사용하면 여러 요청을 **동시에** 처리할 수 있습니다.

In [6]:
import asyncio
import time

questions = [
    {'question': 'Python이 뭔가요?'},
    {'question': 'JavaScript가 뭔가요?'},
    {'question': 'Rust가 뭔가요?'},
]

# 동시 실행 (asyncio.gather)
print("=== asyncio.gather()로 동시 실행 ===")
start = time.time()

results = await asyncio.gather(*[
    chatbot.ainvoke(q) for q in questions
])

print(f"총 소요 시간: {time.time() - start:.2f}초")
print(f"처리된 요청 수: {len(results)}개\n")

for i, resp in enumerate(results):
    print(f"[{i+1}] {resp.content[:80]}...\n")

=== asyncio.gather()로 동시 실행 ===
총 소요 시간: 312.22초
처리된 요청 수: 3개

[1] Python은 프로그래밍 언어입니다. Python은 간단하고 쉽게 understandable한 언어로, 수많은 developers와 scient...

[2] 자바스크립트(JavaScript)은 자바 스크リ프트という 이름으로도 알려져 있으며, World Wide Web의 HTML страниц에 구현된...

[3] Rust는 고성능과 안정성, 그리고 수명이ยาวนาน할 수 있는 소프트웨어를 개발하기 위한 프로그래밍 언어입니다. Rust programming...



# 7. 동기 vs 비동기 성능 비교

여러 요청을 처리할 때 동기와 비동기의 차이를 비교합니다.

In [7]:
# 동기 방식으로 순차 실행
@chain
def sync_chatbot(values):
    prompt = template.invoke(values)
    return model.invoke(prompt)

print("=== 동기 순차 실행 ===")
start = time.time()

sync_results = []
for q in questions:
    sync_results.append(sync_chatbot.invoke(q))

sync_time = time.time() - start
print(f"동기 순차 실행 시간: {sync_time:.2f}초")

# 비동기 동시 실행
print("\n=== 비동기 동시 실행 ===")
start = time.time()

async_results = await asyncio.gather(*[
    chatbot.ainvoke(q) for q in questions
])

async_time = time.time() - start
print(f"비동기 동시 실행 시간: {async_time:.2f}초")

print(f"\n성능 향상: {sync_time / async_time:.1f}배 빠름")

=== 동기 순차 실행 ===
동기 순차 실행 시간: 472.08초

=== 비동기 동시 실행 ===
비동기 동시 실행 시간: 457.41초

성능 향상: 1.0배 빠름


---

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본 (OpenAI)
from langchain_openai.chat_models import ChatOpenAI
model = ChatOpenAI(model='gpt-3.5-turbo')

# 변경 (Ollama)
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2')
```

## 비동기 메서드 정리

| 동기 메서드 | 비동기 메서드 | 용도 |
|------------|--------------|------|
| `invoke()` | `ainvoke()` | 단일 실행 |
| `batch()` | `abatch()` | 배치 실행 |
| `stream()` | `astream()` | 스트리밍 |

## Python 스크립트에서 실행하기

Jupyter가 아닌 일반 스크립트에서는 `asyncio.run()`을 사용합니다:

```python
import asyncio

async def main():
    result = await chatbot.ainvoke({'question': '안녕!'})
    print(result.content)

if __name__ == "__main__":
    asyncio.run(main())
```

## FastAPI에서 사용 예시

```python
from fastapi import FastAPI

app = FastAPI()

@app.post("/chat")
async def chat(question: str):
    response = await chatbot.ainvoke({'question': question})
    return {"answer": response.content}
```